 # SQL Injection Demo

Setting up connection to db

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('demonstration.db')
cursor = conn.cursor()


In [2]:
def print_users(result):
    if (len(result) == 0):
        print("No data retrieved")
    else:
        print(pd.DataFrame(result, columns=["username", "first name", "last name", "password"]))
    

# Example from presentation (Unsafe)

In [7]:
def unsafe_find_user_by_fName(fName):
    query = "SELECT username, fName, lName, password FROM users where fName = '" + fName + "'"
    print("Resulting query: " + query + "\n")
    result = cursor.execute(query).fetchall()
    print_users(result)

## Intended usage

In [8]:
unsafe_find_user_by_fName("David")

Resulting query: SELECT username, fName, lName, password FROM users where fName = 'David'

            username first name last name        password
0  ComputerWhisperer      David    Stotts  i032r23b2o1kwe


## Intended, non malicious usage

In [9]:
unsafe_find_user_by_fName("Dav'id")

Resulting query: SELECT username, fName, lName, password FROM users where fName = 'Dav'id'



OperationalError: near "id": syntax error

## Malicious usage

In [10]:
unsafe_find_user_by_fName("David' OR 1=1--")

Resulting query: SELECT username, fName, lName, password FROM users where fName = 'David' OR 1=1--'

            username first name last name        password
0         CompSciKid      James     Jones      password01
1        GuyWhoCodes     Steven     Smith          123456
2  ComputerWhisperer      David    Stotts  i032r23b2o1kwe
3     TheCodeBringer      Brian  Richards        drowssap


# Preventing SQL Injection

In [13]:
def safe_find_user_by_fName(fName):
    query = "SELECT username, fName, lName, password FROM users where fName =?"
    result = cursor.execute(query, (fName,)).fetchall()
    print_users(result)

# Intended usage

In [14]:
safe_find_user_by_fName("David")

            username first name last name        password
0  ComputerWhisperer      David    Stotts  i032r23b2o1kwe


# Intended, non malicious usage

In [15]:
safe_find_user_by_fName("Dav'id")

No data retrieved


# Malicious usage

In [17]:
safe_find_user_by_fName("David OR 1=1--")

No data retrieved


# Unsafe method

In [ ]:
def unsafe_get_user_by_id(id):
    # String concatenation and substitution are both vulnerable
    query = "SELECT username, fName, lName, password FROM users WHERE uid = " + id
    #query = "SELECT username, fName, lName, password FROM users WHERE uid = %s" % id
    print("Resulting query: " + query + "\n")
    result = cursor.execute(query).fetchall()
    print_users(result)

## Intended usage

In [ ]:
unsafe_get_user_by_id("1")

## Unintended usage

In [ ]:
unsafe_get_user_by_id("1 OR 1=1")






# Safe Method

In [ ]:
def safe_get_user_by_id(id):
    query = "SELECT username, fName, lName, password FROM users WHERE uid = ?"
    result = cursor.execute(query, (id,)).fetchall()
    print_users(result)

# Intended usage

In [ ]:
safe_get_user_by_id("1")

# Unintended usage

In [ ]:
safe_get_user_by_id("1 OR 1=1")